# Demo notebook for using maridatadownloader

# Table of content <a class="anchor" id="toc"></a>

* [1) GFS](#gfs)
    * [1.1) Create downloader object](#gfs-init)
    * [1.2) Download data as datacube](#gfs-datacube)
    * [1.3) Download data along trajectory](#gfs-trajectory)
* [2) CMEMS](#cmems)
    * [1.1) Create downloader object](#cmems-init)
    * [1.2) Download data as datacube](#cmems-datacube)

---

In [ ]:
import os
import time

import getpass
import numpy as np
import rioxarray
import xarray

from maridatadownloader import DownloaderFactory

In [ ]:
%load_ext autoreload
%autoreload 2

---

# 1) GFS <a class="anchor" id="gfs"></a>
[go to TOC](#toc)

## 1.1) Create downloader object <a class="anchor" id="gfs-init"></a>
[go to TOC](#toc)

In [ ]:
gfs = DownloaderFactory.get_downloader('opendap', 'gfs')

In [ ]:
gfs.dataset

## 1.2) Download data as datacube <a class="anchor" id="gfs-datacube"></a>
[go to TOC](#toc)

In [ ]:
# Configuration for downloading as cube
time_min = "2023-07-30 11:00:00"
time_max = "2023-07-30 16:00:00"

lon_min=-180
lon_max=180
lat_min=-90
lat_max=90

height_min = 10
height_max = 10

parameters = ["Temperature_surface", "Pressure_reduced_to_MSL_msl", "Wind_speed_gust_surface", 
              "u-component_of_wind_height_above_ground", "v-component_of_wind_height_above_ground"]

sel_dict = {'time': slice(time_min, time_max), 'time1': slice(time_min, time_max)}
#sel_dict = {'time': slice(time_min, time_max), 'time1': slice(time_min, time_max), 
#            'height_above_ground2': slice(height_min, height_max)}

In [ ]:
dataset_gfs = gfs.download(parameters=parameters, sel_dict=sel_dict)
dataset_gfs

In [ ]:
if 'sigma' in dataset_gfs.coords:
    dataset_gfs = dataset_gfs.isel(sigma=0).drop('sigma')
if 'height_above_ground' in dataset_gfs.coords and 'height_above_ground' not in sel_dict.keys():
    dataset_gfs = dataset_gfs.sel(height_above_ground=10).drop('height_above_ground')
if 'reftime' in dataset_gfs.coords:
    dataset_gfs = dataset_gfs.drop('reftime')
if 'LatLon_Projection' in dataset_gfs.coords:   
    dataset_gfs = dataset_gfs.drop('LatLon_Projection')
dataset_gfs

In [ ]:
gfs.dataset['u-component_of_wind_height_above_ground'].sel(height_above_ground2=10).isel(time=0).plot()

### Calculate difference 

In [ ]:
diff = gfs.dataset['u-component_of_wind_height_above_ground'].sel(height_above_ground2=10).isel(time=0) - gfs.dataset['u-component_of_wind_height_above_ground'].sel(height_above_ground2=80).isel(time=0)

In [ ]:
diff.plot()

## 1.3) Download data along trajectory <a class="anchor" id="gfs-trajectory"></a>
[go to TOC](#toc)

In [ ]:
# Configuration for downloading along trajectory
# https://stackoverflow.com/questions/72179103/xarray-select-the-data-at-specific-x-and-y-coordinates
lons = [2.81, 3.19, 4.56, 6.67, 6.68]
lats = [51.9, 53.0, 54.0, 54.3, 55.5]
times = ["2023-07-30 09:00:00",
         "2023-07-30 11:00:00",
         "2023-07-30 13:00:00",
         "2023-07-30 15:00:00",
         "2023-07-30 17:00:00"]

parameters = ["Temperature_surface", "Pressure_reduced_to_MSL_msl", "Wind_speed_gust_surface", 
              "u-component_of_wind_height_above_ground", "v-component_of_wind_height_above_ground"]
                 
x = xarray.DataArray(lons, dims=['trajectory'])
y = xarray.DataArray(lats, dims=['trajectory'])
t = xarray.DataArray(times, dims=['trajectory'])

sel_dict = {'time': t, 'time1': t, 'lon': x, 'lat': y}

In [ ]:
ds_trajectory = gfs.dataset.sel(sel_dict, method="backfill")
ds_trajectory

In [ ]:
ds_trajectory['trajectory']

In [ ]:
ds_trajectory['Temperature_surface'].plot()

---

# 2) CMEMS <a class="anchor" id="cmems"></a>
[go to TOC](#toc)

## 2.1) Create downloader object <a class="anchor" id="cmems-init"></a>
[go to TOC](#toc)

In [ ]:
USERNAME = '<insert-username>'
PASSWORD = getpass.getpass('Enter your password: ')

In [ ]:
#product = 'cmems_mod_glo_phy_anfc_merged-uv_PT1H-i'  # currents
#product = 'cmems_mod_glo_phy_anfc_0.083deg_PT1H-m'  # phyiscs
product = 'cmems_mod_glo_phy_my_0.083_P1D-m' # phyiscs, my
#product = 'cmems_mod_glo_wav_anfc_0.083deg_PT3H-i'  # waves
#product = 'cmems_obs-wind_glo_phy_nrt_l4_0.125deg_PT1H' # wind
product_type = 'my'

In [ ]:
downloader_kwargs = {'product': product, 'product_type': product_type}

In [ ]:
cmems = DownloaderFactory.get_downloader('opendap', 'cmems', USERNAME, PASSWORD, **downloader_kwargs)

## 2.2) Download data as datacube <a class="anchor" id="cmems-datacube"></a>
[go to TOC](#toc)

In [ ]:
time_min = "2023-05-24T12:00:00"
time_max = "2023-05-24T18:00:00"

lon_min=-180
lon_max=180
lat_min=-90
lat_max=90

parameters = ["VHM0", "VMDR", "VTPK"]
#parameters = ["uo","vo","vsdx","vsdy","utide","vtide","utotal","vtotal"]
#parameters = ["so"]

sel_dict = {'time': slice(time_min, time_max)}

In [ ]:
dataset_cmemcs = cmems.download(parameters=parameters, sel_dict=sel_dict)
dataset_cmemcs

### Check ocean currents contributions

In [ ]:
diff = dataset_cmemcs['uo'].isel(time=0) + dataset_cmemcs['vsdx'].isel(time=0) + dataset_cmemcs['utide'].isel(time=0) - dataset_cmemcs['utotal'].isel(time=0)

In [ ]:
diff.plot()